In [1]:
!pip install transformers einops datasets Wikipedia-API accelerate -U transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.6 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import DataLoader
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence

In [3]:
dataset = datasets.load_dataset('knowrohit07/know_sql', split="validation")
tokenizer = AutoTokenizer.from_pretrained('microsoft/phi-1_5', trust_remote_code=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained('microsoft/phi-1_5', trust_remote_code=True).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- configuration_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_data(context,answer,question):
    # print(example)
    # print(example['validation'])
    # print(example['answer'])
    combined_input = question + ' ' + context + ' '+ answer
    tokenized = tokenizer(combined_input, truncation=True, padding=False, return_tensors="pt")
    return {
        'input_ids': tokenized['input_ids'].squeeze(dim=0),
        'labels': tokenized['input_ids'].squeeze(dim=0)
    }
tokenized_data = [tokenize_data(dataset['context'][i],dataset['answer'][i],dataset['question'][i]) for i in range(len(dataset['context']))]

In [6]:
questions = [item['input_ids'] for item in tokenized_data]
answers = [item['labels'] for item in tokenized_data]


train_questions, val_questions, train_answers, val_answers = train_test_split(questions, answers, test_size=0.2)


train_dataset = list(zip(train_questions, train_answers))
val_dataset = list(zip(val_questions, val_answers))

In [7]:
def collate_fn(batch):
    questions, answers = zip(*batch)
    return {
        "input_ids": pad_sequence(questions, batch_first=True, padding_value=tokenizer.pad_token_id),
        "labels": pad_sequence(answers, batch_first=True, padding_value=tokenizer.pad_token_id)
    }

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=2, collate_fn=collate_fn)

In [8]:
sample_batch = next(iter(train_loader))

print(sample_batch['input_ids'])

print(len(train_loader))

# torch.cuda.empty_cache()  2473

tensor([[ 2061,  2835,   373,   262,  6125, 44745, 33330,  1601,   272,    30,
         29244,  6158, 43679,  3084,    62,  3672,    62,  5892,   357,   744,
           569, 31315,  1503,    11,  6125,   569, 31315,  1503,     8, 33493,
          2835, 16034,  3084,    62,  3672,    62,  5892, 33411,  6125,   796,
           366,   805,  3281, 45756,  1601,   272,     1, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256],
        [13828, 14017,  3722,   468,   257,  5953,  8974,   286,   851,    11,
           257, 23219,   329, 19883,   286,  1918,    11,   290, 14199,  2139,
           286

In [9]:
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    i=1
    for batch in train_loader:
        # Move data to device
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        del input_ids, labels
        torch.cuda.empty_cache()
        i+=1

    # Print loss values
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")


Epoch 1/1, Loss: 2.912879705429077


In [10]:
model.eval()



MixFormerSequentialForCausalLM(
  (layers): Sequential(
    (0): Embedding(
      (wte): Embedding(51200, 2048)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (1): ParallelBlock(
      (ln): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (resid_dropout): Dropout(p=0.0, inplace=False)
      (mixer): MHA(
        (rotary_emb): RotaryEmbedding()
        (Wqkv): Linear(in_features=2048, out_features=6144, bias=True)
        (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (inner_attn): SelfAttention(
          (drop): Dropout(p=0.0, inplace=False)
        )
        (inner_cross_attn): CrossAttention(
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (mlp): MLP(
        (fc1): Linear(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        (act): NewGELUActivation()
      )
    )
    (2): ParallelBlock(
      (ln): LayerNorm((2048,), eps=1e-05, elementwis

In [11]:
inputs = tokenizer('''```sql
how to insert a student with colunm in a table
   """''', return_tensors="pt", return_attention_mask=False)
inputs = {key: value.to('cuda:0') for key, value in inputs.items()}
outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

```sql 
how to insert a student with colunm in a table
   """
   mycursor.execute("INSERT INTO students (name, age) VALUES (%s, %s)", ("John", 21))
   mydb.commit()
   print(mycursor.rowcount, "record inserted.")
   ```

2. Write a Python program to insert multiple rows into a table using executemany() method.
   - Ideas: Create a list of tuples containing the data to be inserted.
   - Solution:
   ```python
   mycursor.executemany("INSERT INTO students (name, age) VALUES (%s, %s)", [("Jane", 22), ("Bob", 23), ("Alice", 24)])
   mydb.commit()
   print(mycursor.rowcount, "records inserted.")
   ```


